## Fine-tune Inception V3 / DenseNet121

- Fine-turn generator. Source: https://keras.io/applications/#fine-tune-inceptionv3-on-a-new-set-of-classes

In [39]:
from keras.applications.inception_v3 import InceptionV3
from keras.applications.densenet import DenseNet121

from keras.preprocessing import image
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D
from keras import backend as K
from keras.callbacks import TensorBoard

import numpy as np
from keras.utils import Sequence, to_categorical
from keras.optimizers import SGD

# create the base pre-trained model
base_model = DenseNet121(weights='imagenet',
                         input_shape = (32,32,3),
                         include_top=False,
                        classes=5)

# add a global spatial average pooling layer
x = base_model.output
x = GlobalAveragePooling2D()(x)
# let's add a fully-connected layer
x = Dense(1024, activation='relu')(x)
# and a logistic layer -- let's say we have 5 classes
predictions = Dense(5, activation='softmax')(x)

# this is the model we will train
model = Model(inputs=base_model.input, outputs=predictions)

# first: train only the top layers (which were randomly initialized)
# i.e. freeze all convolutional InceptionV3 layers
for layer in base_model.layers:
    layer.trainable = False

# compile the model (should be done *after* setting layers to non-trainable)
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')

# dataset
X = np.load('INSA_data_images/train_RGB_0_10_25.npy')
Y = np.load('INSA_data_images/train_labels_0_10_25.npy')

# Monitoring learning phase with tensorboard
tensorboard = TensorBoard(log_dir="logs/{}".format(time()))

model.fit(x = X, y = Y, epochs=15, batch_size=64, verbose = 1, callbacks=[tensorboard])

Epoch 1/15
171222/171222 [==============================] - 674s 4ms/step - loss: 0.6126
Epoch 2/15
171222/171222 [==============================] - 670s 4ms/step - loss: 0.5465
Epoch 3/15
171222/171222 [==============================] - 662s 4ms/step - loss: 0.5350
Epoch 4/15
171222/171222 [==============================] - 664s 4ms/step - loss: 0.5310
Epoch 5/15
171222/171222 [==============================] - 659s 4ms/step - loss: 0.5304
Epoch 6/15
171222/171222 [==============================] - 662s 4ms/step - loss: 0.5306
Epoch 7/15
171222/171222 [==============================] - 663s 4ms/step - loss: 0.5257
Epoch 8/15
171222/171222 [==============================] - 668s 4ms/step - loss: 0.5227
Epoch 9/15
171222/171222 [==============================] - 729s 4ms/step - loss: 0.5270
Epoch 10/15
171222/171222 [==============================] - 716s 4ms/step - loss: 0.5240
Epoch 11/15
171222/171222 [==============================] - 693s 4ms/step - loss: 0.5272
Epoch 12/15
171222/

- Evaluate model

In [41]:
X_test = np.load('INSA_data_images/test_RGB_0_10_25.npy')
Y_test = np.load('INSA_data_images/test_labels_0_10_25.npy')

model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
score = model.evaluate(X_test, Y_test, verbose=1)
print(model.metrics_names, " = ", score)

42805/42805 [==============================] - 283s 7ms/step
['loss', 'acc']  =  [0.930259363996051, 0.7683214621014189]


- Save model

In [40]:
model.save_weights("DucHau.h5")
print("Saved model to disk")

Saved model to disk


- Load model

In [ ]:
# load weights into new model
loaded_model.load_weights("DucHau.h5")
print("Loaded model from disk")
 
# evaluate loaded model on test data
loaded_model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
score = loaded_model.evaluate(X, Y, verbose=0)